In [1]:
import pandas as pd
import numpy as np
import os

In [2]:
data = pd.read_parquet('codedData.parquet')

In [8]:
#group data by s_residence and sum weight_peak
print(data.groupby(['s_hisp']).sum()['weight_peak'].reset_index())
# calculate the percentage of each group dont include nulls
round(data.groupby(['s_hisp']).sum()['weight_peak'].reset_index()['weight_peak']/data.groupby(['s_hisp']).sum()['weight_peak'].reset_index()['weight_peak'].sum(), 2)

  s_hisp  weight_peak
0     No       251.66
1    Yes         9.60


0    0.96
1    0.04
Name: weight_peak, dtype: float64

In [3]:
from transformers import AutoTokenizer, AutoModel
import torch
import torch.nn.functional as F

#Mean Pooling - Take attention mask into account for correct averaging
def mean_pooling(model_output, attention_mask):
    token_embeddings = model_output[0] #First element of model_output contains all token embeddings
    input_mask_expanded = attention_mask.unsqueeze(-1).expand(token_embeddings.size()).float()
    return torch.sum(token_embeddings * input_mask_expanded, 1) / torch.clamp(input_mask_expanded.sum(1), min=1e-9)


# Sentences we want sentence embeddings for
sentences = ['This is an example sentence', 'Each sentence is converted']

# Load model from HuggingFace Hub
tokenizer = AutoTokenizer.from_pretrained('sentence-transformers/all-mpnet-base-v2')
model = AutoModel.from_pretrained('sentence-transformers/all-mpnet-base-v2')

print(model.name_or_path)

# Tokenize sentences
encoded_input = tokenizer(sentences, padding=True, truncation=True, return_tensors='pt')

# Compute token embeddings
with torch.no_grad():
    model_output = model(**encoded_input)

# Perform pooling
sentence_embeddings = mean_pooling(model_output, encoded_input['attention_mask'])

# Normalize embeddings
sentence_embeddings = F.normalize(sentence_embeddings, p=2, dim=1)

print("Sentence embeddings:")
print(sentence_embeddings)

sentence-transformers/all-mpnet-base-v2
Sentence embeddings:
tensor([[ 0.0225, -0.0783, -0.0230,  ..., -0.0083,  0.0265, -0.0020],
        [ 0.0417,  0.0011, -0.0155,  ..., -0.0218, -0.0636, -0.0088]])
